In [ ]:
%%capture
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
log = logging.getLogger()
log.setLevel(logging.DEBUG)

import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
%cd /home/schirrmr/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
from hyperoptim.results import load_data_frame, remove_columns_with_same_value, dataset_averaged_frame
from hyperoptim.results import mean_identical_exps
import pandas as pd
from hyperoptim.results import remove_columns_with_same_value, remove_indices_with_same_value

import json
from collections import OrderedDict

from copy import deepcopy
from glob import glob 

In [ ]:
entry[0][0]

In [ ]:
all_entries[0]

In [ ]:
len(all_entries)

In [ ]:
folder = 'data/models/pytorch/auto-diag/smac-shallow//'
runhistory_folders = glob(os.path.join(folder,'deterministic_run*/'))
entry_numbers = []
run_ids = sorted([int(f.split('run')[-1][:-1]) for f in runhistory_folders])
for run_id in run_ids:
    print(run_id)
    trac_filename = os.path.join(
        folder, 'deterministic_run{:d}/traj_aclib2.json'.format(run_id))
    content = open(trac_filename, 'r').read()
    print("len", len(content.split('\n')))

In [ ]:
traj_json

In [ ]:
folder = 'data/models/pytorch/auto-diag/smac-deep4/'
runhistory_folders = glob(os.path.join(folder,'deterministic_run*/'))
entry_numbers = []
run_ids = sorted([int(f.split('run')[-1][:-1]) for f in runhistory_folders])
for run_id in run_ids:
    runhistory_filename = os.path.join(
        folder, 'deterministic_run{:d}/runhistory.json'.format(run_id))
    content = open(runhistory_filename, 'r').read()
    runjson = json.loads(content)
    #print(run_id)
    this_entry_nums = [entry[0][0] for entry in runjson['data']]
    entry_numbers.append(this_entry_nums)

## Find reason discrepancy my statistics best smac

In [ ]:
folder = 'data/models/pytorch/auto-diag/smac-shallow/'

runhistory_filename = os.path.join(
        folder, 'deterministic_run8/runhistory.json'.format(run_id))
content = open(runhistory_filename, 'r').read()
runjson = json.loads(content)

In [ ]:
entries = runjson['data']
configs = runjson['configs']

In [ ]:
len([e for e in entries if e[0][0] == 364])

In [ ]:
[e for e in entries if e[0][0] == 364]

In [ ]:
ent

## Run statistics



In [ ]:
all_entries = []
enum_crashed = 'StatusType.CRASHED'
enum_success = 'StatusType.SUCCESS'
enum_timeout = 'StatusType.TIMEOUT'
specific_entries = {enum_crashed:[], enum_success: [], enum_timeout: []}
folder = 'data/models/pytorch/auto-diag/smac-deep4/'
runhistory_folders = glob(os.path.join(folder,'deterministic_run*/'))
run_ids = sorted([int(f.split('run')[-1][:-1]) for f in runhistory_folders])
for run_id in run_ids:
    runhistory_filename = os.path.join(
        folder, 'deterministic_run{:d}/runhistory.json'.format(run_id))
    content = open(runhistory_filename, 'r').read()
    runjson = json.loads(content)
    #print(run_id)
    for entry in runjson['data']:
        assert entry[1][1] != 1234567890
        outcome = entry[1][2]
        assert outcome['__enum__'] in (enum_crashed,enum_success,enum_timeout)
        specific_entries[outcome['__enum__']].append(tuple(entry[0]))
        all_entries.append(tuple(entry[0]))
    #print("len entry", len(runjson['data']))

n_all_runs= len(all_entries)
n_unique_runs= len(set(all_entries))
n_unique_configs = len(set([e[0] for e in all_entries]))
print("all entries", n_all_runs)
print("unique configs", n_unique_configs)

for key in specific_entries:
    n_runs = len(specific_entries[key])
    print(key, n_runs, np.round(100 *  n_runs/ float(n_all_runs),0))
    n_this_unique_configs = len(np.unique([config for config, fold, _ in specific_entries[key]]))
    print(key, n_this_unique_configs,
          np.round(100 * n_this_unique_configs / float(n_unique_configs)))

    
successes = set([e[0] for e in specific_entries['StatusType.SUCCESS']])
timeouts = set([e[0] for e in specific_entries['StatusType.TIMEOUT']])
crashes = set([e[0] for e in specific_entries['StatusType.CRASHED']])
all_successful = np.setdiff1d(np.setdiff1d(list(successes), list(timeouts)), list(crashes))
all_timeouts = np.setdiff1d(np.setdiff1d(list(timeouts), list(successes)), list(crashes))
all_crashes = np.setdiff1d(np.setdiff1d(list(crashes), list(successes)), list(timeouts))
print("all success", len(all_successful), 
       np.round(100 * len(all_successful) / float(len(set([e[0] for e in all_entries]))),0))
all_successful = np.setdiff1d(np.setdiff1d(list(successes), list(timeouts)), list(crashes))
print("all timeout", len(all_timeouts), 
       np.round(100 * len(all_timeouts) / float(len(set([e[0] for e in all_entries]))),0))
all_successful = np.setdiff1d(np.setdiff1d(list(successes), list(timeouts)), list(crashes))
print("all crashes", len(all_crashes), 
       np.round(100 * len(all_crashes) / float(len(set([e[0] for e in all_entries]))),0))

In [ ]:
all_entries = []
enum_crashed = 'StatusType.CRASHED'
enum_success = 'StatusType.SUCCESS'
enum_timeout = 'StatusType.TIMEOUT'
specific_entries = {enum_crashed:[], enum_success: [], enum_timeout: []}
folder = 'data/models/pytorch/auto-diag/smac-shallow//'
runhistory_folders = glob(os.path.join(folder,'deterministic_run*/'))
run_ids = sorted([int(f.split('run')[-1][:-1]) for f in runhistory_folders])
for run_id in run_ids:
    runhistory_filename = os.path.join(
        folder, 'deterministic_run{:d}/runhistory.json'.format(run_id))
    content = open(runhistory_filename, 'r').read()
    runjson = json.loads(content)
    #print(run_id)
    for entry in runjson['data']:
        assert entry[1][1] != 1234567890
        outcome = entry[1][2]
        assert outcome['__enum__'] in (enum_crashed,enum_success,enum_timeout)
        specific_entries[outcome['__enum__']].append(tuple(entry[0]))
        all_entries.append(tuple(entry[0]))
    #print("len entry", len(runjson['data']))

n_all_runs= len(all_entries)
n_unique_runs= len(set(all_entries))
n_unique_configs = len(set([e[0] for e in all_entries]))
print("all entries", n_all_runs)
print("unique configs", n_unique_configs)

for key in specific_entries:
    n_runs = len(specific_entries[key])
    print(key, n_runs, np.round(100 *  n_runs/ float(n_all_runs),0))
    n_this_unique_configs = len(np.unique([config for config, fold, _ in specific_entries[key]]))
    print(key, n_this_unique_configs,
          np.round(100 * n_this_unique_configs / float(n_unique_configs)))

    
successes = set([e[0] for e in specific_entries['StatusType.SUCCESS']])
timeouts = set([e[0] for e in specific_entries['StatusType.TIMEOUT']])
crashes = set([e[0] for e in specific_entries['StatusType.CRASHED']])
all_successful = np.setdiff1d(np.setdiff1d(list(successes), list(timeouts)), list(crashes))
all_timeouts = np.setdiff1d(np.setdiff1d(list(timeouts), list(successes)), list(crashes))
all_crashes = np.setdiff1d(np.setdiff1d(list(crashes), list(successes)), list(timeouts))
print("all success", len(all_successful), 
       np.round(100 * len(all_successful) / float(len(set([e[0] for e in all_entries]))),0))
all_successful = np.setdiff1d(np.setdiff1d(list(successes), list(timeouts)), list(crashes))
print("all timeout", len(all_timeouts), 
       np.round(100 * len(all_timeouts) / float(len(set([e[0] for e in all_entries]))),0))
all_successful = np.setdiff1d(np.setdiff1d(list(successes), list(timeouts)), list(crashes))
print("all crashes", len(all_crashes), 
       np.round(100 * len(all_crashes) / float(len(set([e[0] for e in all_entries]))),0))